In [ ]:
import pandas as pd
import numpy as np
import os

os.chdir("/content/drive/MyDrive/ML 대회/HDD")
df = pd.read_csv("./test.csv",parse_dates=["assembly_time","process_time"])

from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df['cr_ID'] = le.fit_transform(df['cr_ID'])
# df['eq_ID'] = le.fit_transform(df['eq_ID'])

# 범주형 변수
categorical_feature_names = ['ID','cr_ID', 'eq_ID' ]

for var in categorical_feature_names:
    df[var] = df[var].astype("category")  #카테고리화

# 조립 시작 시간
df["assem_year"] = df["assembly_time"].dt.year
df["assem_month"] = df["assembly_time"].dt.month
df["assem_day"] = df["assembly_time"].dt.day
df["assem_hour"] = df["assembly_time"].dt.hour
df["assem_min"] = df["assembly_time"].dt.minute
df["assem_sec"] = df["assembly_time"].dt.second

# 후공정 시작 시간
df["pro_year"] = df["process_time"].dt.year
df["pro_month"] = df["process_time"].dt.month
df["pro_day"] = df["process_time"].dt.day
df["pro_hour"] = df["process_time"].dt.hour
df["pro_min"] = df["process_time"].dt.minute
df["pro_sec"] = df["process_time"].dt.second

from sklearn import preprocessing
df['ramp_scale'] = preprocessing.robust_scale(df['ramp'])
df['servo_scale'] = preprocessing.robust_scale(df['servo_defect'])
df["data_scale"] = preprocessing.robust_scale(df['data_defect'])

# 후공정 시작 시간 - 조립 시작 시간
df["pro_assem"] = df["process_time"] - df["assembly_time"]
df["pro_assem"] = df["pro_assem"].dt.total_seconds()  # 초 단위로 변환

# categorical: cr_ID, eq_ID
cr_enc = df['cr_ID'].value_counts().to_dict()
df['cr_ID_counts'] = df['cr_ID'].map(cr_enc)
eq_enc = df['eq_ID'].value_counts().to_dict()
df['eq_ID_counts'] = df['eq_ID'].map(eq_enc)

# cr_ID Aggregation
tmp1 = df.groupby('cr_ID')['ramp'].agg(['mean']).rename({'mean':'ramp_cr_mean'},axis=1)
tmp2 = df.groupby('cr_ID')['servo_defect'].agg(['mean']).rename({'mean':'servo_cr_mean'},axis=1)
tmp3 = df.groupby('cr_ID')['data_defect'].agg(['mean']).rename({'mean':'data_cr_mean'},axis=1)
tmp4 = df.groupby('cr_ID')['pro_assem'].agg(['mean']).rename({'mean':'cr_proassem'},axis=1)
df = pd.merge(df,tmp1,on='cr_ID',how='left')
df = pd.merge(df,tmp2,on='cr_ID',how='left')
df = pd.merge(df,tmp3,on='cr_ID',how='left')
df = pd.merge(df,tmp4,on='cr_ID',how='left')

# eq_ID Aggregation
tmp1 = df.groupby('eq_ID')['ramp'].agg(['mean']).rename({'mean':'ramp_eq_mean'},axis=1)
tmp2 = df.groupby('eq_ID')['servo_defect'].agg(['mean']).rename({'mean':'servo_eq_mean'},axis=1)
tmp3 = df.groupby('eq_ID')['data_defect'].agg(['mean']).rename({'mean':'data_eq_mean'},axis=1)
tmp4 = df.groupby('eq_ID')['pro_assem'].agg(['mean']).rename({'mean':'eq_proassem'},axis=1)
df = pd.merge(df,tmp1,on='eq_ID',how='left')
df = pd.merge(df,tmp2,on='eq_ID',how='left')
df = pd.merge(df,tmp3,on='eq_ID',how='left')
df = pd.merge(df,tmp4,on='eq_ID',how='left')

# pro_ID&ramp Aggregation
tmp1 = df.groupby('pro_day')['ramp'].agg(['sum']).rename({'sum':'ramp_pro_sum'},axis=1)
tmp2 = df.groupby('pro_day')['ramp'].agg(['mean']).rename({'mean':'ramp_pro_mean'},axis=1)
df = pd.merge(df,tmp1,on='pro_day',how='left')
df = pd.merge(df,tmp2,on='pro_day',how='left')

df["defect_all"] = df["servo_defect"]+df["data_defect"] #총 bad sector 개수
df["defect_minus"]= abs(df["servo_defect"]-df["data_defect"])   #bad sector 차이
df["ramp_proassem"] = df["ramp"]/df["pro_assem"]  #ramp ratio

In [ ]:
df[features].head()

,cr_ID,eq_ID,cr_ID_counts,eq_ID_counts,ramp_scale,servo_scale,data_scale,assem_hour,assem_min,pro_day,pro_hour,pro_min,pro_sec,pro_assem,ramp_cr_mean,servo_cr_mean,data_cr_mean,ramp_eq_mean,servo_eq_mean,data_eq_mean,cr_proassem,eq_proassem,defect_all,defect_minus,ramp_proassem,ramp_pro_sum,ramp_pro_mean
0,CRSTR01A,EGRWT0107,398,528,-0.093837,1.133355,-0.489129,11,47,8,12,13,12,1572.0,2530.701005,4107.288945,2893.055276,2441.530303,4243.636364,2985.066288,5624.452261,5913.657197,7404.0,4164.0,0.965649,2926480.0,2499.128950
1,CRSTR01O,EGRWT0113,369,564,-0.331345,0.615739,0.488626,10,14,11,12,3,49,6589.0,2440.330623,4183.943089,3039.140921,2627.303191,4202.166667,3076.070922,5665.441734,5784.723404,8563.0,1433.0,0.162392,3011202.0,2586.943299
2,CRSTR01P,EGRWT0104,406,527,-0.364215,-0.165295,0.083951,4,32,8,5,20,31,2911.0,2631.337438,4149.620690,2898.724138,2403.113852,4220.419355,3026.223909,5736.081281,5911.180266,6572.0,1052.0,0.346273,2926480.0,2499.128950
3,CRSTR01I,EGRWT0114,378,556,-0.346720,1.030622,-0.625361,19,13,9,20,37,23,5063.0,2502.658730,4179.238095,2900.685185,2492.106115,4183.433453,3008.370504,5673.230159,5974.845324,6977.0,4279.0,0.205609,2749554.0,2424.650794
4,CRSTR01R,EGRWT0111,425,512,0.241219,0.057293,0.558502,17,33,7,18,16,33,2613.0,2636.458824,4178.632941,2920.343529,2465.906250,4234.597656,2970.722656,5774.564706,5932.097656,7854.0,446.0,0.822809,2903710.0,2505.358067


In [ ]:
import pickle

# features = ['cr_ID','eq_ID','cr_ID_counts','eq_ID_counts','ramp_scale','servo_scale','data_scale',"assem_hour","assem_min","pro_day","pro_hour","pro_min","pro_sec",'pro_assem',
# 'ramp_cr_mean',	'servo_cr_mean','data_cr_mean',	'ramp_eq_mean',	'servo_eq_mean','data_eq_mean', 'cr_proassem','eq_proassem'
# ,"defect_all","defect_minus","ramp_proassem",'ramp_pro_sum','ramp_pro_mean']

features = ['cr_ID_counts','eq_ID_counts','ramp_scale','servo_scale','data_scale',"assem_hour","assem_min","pro_day","pro_hour","pro_min","pro_sec",'pro_assem',
'ramp_cr_mean',	'servo_cr_mean','data_cr_mean',	'ramp_eq_mean',	'servo_eq_mean','data_eq_mean', 'cr_proassem','eq_proassem'
,"defect_all","defect_minus","ramp_proassem",'ramp_pro_sum','ramp_pro_mean']

model = pickle.load(open('./model/rf_dfver4-no-le.sav', 'rb'))
prediction = model.predict(df[features])
submission = pd.DataFrame(prediction, columns=["status"])
submission.head()

,status
0,pass
1,pass
2,pass
3,pass
4,pass


In [ ]:
submission.to_csv("./Submission/submission_dfver4.csv", index=False)

### Test Set 세팅

In [ ]:
import pandas as pd
import numpy as np
import os

os.chdir("/content/drive/MyDrive/ML 대회/HDD")
df = pd.read_csv("./test.csv",parse_dates=["assembly_time","process_time"])

from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df['cr_ID'] = le.fit_transform(df['cr_ID'])
# df['eq_ID'] = le.fit_transform(df['eq_ID'])

# 범주형 변수
categorical_feature_names = ['ID','cr_ID', 'eq_ID' ]

for var in categorical_feature_names:
    df[var] = df[var].astype("category")  #카테고리화

# 조립 시작 시간
df["assem_year"] = df["assembly_time"].dt.year
df["assem_month"] = df["assembly_time"].dt.month
df["assem_day"] = df["assembly_time"].dt.day
df["assem_hour"] = df["assembly_time"].dt.hour
df["assem_min"] = df["assembly_time"].dt.minute
df["assem_sec"] = df["assembly_time"].dt.second

# 후공정 시작 시간
df["pro_year"] = df["process_time"].dt.year
df["pro_month"] = df["process_time"].dt.month
df["pro_day"] = df["process_time"].dt.day
df["pro_hour"] = df["process_time"].dt.hour
df["pro_min"] = df["process_time"].dt.minute
df["pro_sec"] = df["process_time"].dt.second

from sklearn import preprocessing
df['ramp_scale'] = preprocessing.robust_scale(df['ramp'])
df['servo_scale'] = preprocessing.robust_scale(df['servo_defect'])
df["data_scale"] = preprocessing.robust_scale(df['data_defect'])

# 후공정 시작 시간 - 조립 시작 시간
df["pro_assem"] = df["process_time"] - df["assembly_time"]
df["pro_assem"] = df["pro_assem"].dt.total_seconds()  # 초 단위로 변환


In [ ]:
## cr_ID 참조 feature 가져오기
df_tr = pd.read_csv("./preprocessed/df_ver4-no-le.csv")
df_tr["cr_ID"] = df_tr["cr_ID"].astype("category")

mapping = dict(df_tr[['cr_ID', 'cr_ID_counts']].values)
df['cr_ID_counts'] = df_tr.cr_ID.map(mapping).astype(int)
mapping = dict(df_tr[['cr_ID', 'ramp_cr_mean']].values)
df['ramp_cr_mean'] = df_tr.cr_ID.map(mapping).astype(float)
mapping = dict(df_tr[['cr_ID', 'servo_cr_mean']].values)
df['servo_cr_mean'] = df_tr.cr_ID.map(mapping).astype(float)
mapping = dict(df_tr[['cr_ID', 'data_cr_mean']].values)
df['data_cr_mean'] = df_tr.cr_ID.map(mapping).astype(float)
mapping = dict(df_tr[['cr_ID', 'cr_proassem']].values)
df['cr_proassem'] = df_tr.cr_ID.map(mapping).astype(float)

## eq_ID 참조 feature 가져오기
df_tr["eq_ID"] = df_tr["eq_ID"].astype("category")

mapping = dict(df_tr[['eq_ID', 'eq_ID_counts']].values)
df['eq_ID_counts'] = df_tr.eq_ID.map(mapping).astype(int)
mapping = dict(df_tr[['eq_ID', 'ramp_eq_mean']].values)
df['ramp_eq_mean'] = df_tr.eq_ID.map(mapping).astype(float)
mapping = dict(df_tr[['eq_ID', 'servo_eq_mean']].values)
df['servo_eq_mean'] = df_tr.eq_ID.map(mapping).astype(float)
mapping = dict(df_tr[['eq_ID', 'data_eq_mean']].values)
df['data_eq_mean'] = df_tr.eq_ID.map(mapping).astype(float)
mapping = dict(df_tr[['eq_ID', 'eq_proassem']].values)
df['eq_proassem'] = df_tr.eq_ID.map(mapping).astype(float)

## pro_day 참조 feature 가져오기
df_tr['pro_day'] = df_tr['pro_day'].astype("category")

mapping = dict(df_tr[['pro_day', 'ramp_pro_sum']].values)
df['ramp_pro_sum'] = df_tr.pro_day.map(mapping).astype(float)
mapping = dict(df_tr[['pro_day', 'ramp_pro_mean']].values)
df['ramp_pro_mean'] = df_tr.pro_day.map(mapping).astype(float)

df["defect_all"] = df["servo_defect"]+df["data_defect"] #총 bad sector 개수
df["defect_minus"]= abs(df["servo_defect"]-df["data_defect"])   #bad sector 차이
df["ramp_proassem"] = df["ramp"]/df["pro_assem"]  #ramp ratio

In [ ]:
features = np.delete(df_tr.columns.values,[len(df_tr.columns.values)-1,len(df_tr.columns.values)-2,len(df_tr.columns.values)-3])
df[features].head()

,cr_ID_counts,eq_ID_counts,ramp_scale,servo_scale,data_scale,assem_hour,assem_min,pro_day,pro_hour,pro_min,pro_sec,pro_assem,ramp_cr_mean,servo_cr_mean,data_cr_mean,ramp_eq_mean,servo_eq_mean,data_eq_mean,cr_proassem,eq_proassem,defect_all,defect_minus,ramp_proassem,ramp_pro_sum,ramp_pro_mean
0,1602,2103,-0.093837,1.133355,-0.489129,11,47,8,12,13,12,1572.0,2466.691011,4178.242821,3012.568664,2415.000951,4205.032335,3011.637660,5818.566167,5973.025202,7404.0,4164.0,0.965649,11254793.0,2479.029295
1,1600,2148,-0.331345,0.615739,0.488626,10,14,11,12,3,49,6589.0,2448.328125,4206.899375,3051.370000,2502.050745,4204.669926,3020.332868,5846.791875,6082.363128,8563.0,1433.0,0.162392,11484543.0,2478.321752
2,1632,2144,-0.364215,-0.165295,0.083951,4,32,8,5,20,31,2911.0,2453.363971,4109.984069,2993.390931,2453.709888,4214.579291,3025.781716,5700.965686,5935.769123,6572.0,1052.0,0.346273,11192258.0,2449.607792
3,1602,2139,-0.346720,1.030622,-0.625361,19,13,9,20,37,23,5063.0,2445.822097,4180.051186,2956.224719,2493.205704,4203.809724,2987.555400,5829.666667,5984.484806,6977.0,4279.0,0.205609,11506082.0,2539.413375
4,1588,2103,0.241219,0.057293,0.558502,17,33,7,18,16,33,2613.0,2462.474811,4150.562972,2975.128463,2415.000951,4205.032335,3011.637660,5864.806675,5973.025202,7854.0,446.0,0.822809,11254793.0,2479.029295


In [ ]:
import pickle ##Ver3

features = ['cr_ID','eq_ID','ramp_scale','servo_scale','data_scale',"assem_hour","assem_min","pro_day","pro_hour","pro_min","pro_sec",'pro_assem',
'ramp_cr_mean',	'servo_cr_mean','data_cr_mean',	'ramp_eq_mean',	'servo_eq_mean','data_eq_mean', 'cr_proassem','eq_proassem'
,"defect_all","defect_minus"]

model = pickle.load(open('./model/rf_dfver3.sav', 'rb'))
prediction = model.predict(df[features])
submission = pd.DataFrame(prediction, columns=["status"])
submission.head()

,status
0,pass
1,pass
2,pass
3,pass
4,fail


In [ ]:
submission.to_csv("./Submission/submission_dfver4-no-le-real.csv", index=False)

In [ ]:
""